###### 2021-03-22

# 01_Iterator
- 값을 차레대로 꺼낼 수 았는 객체
- `for i in range(100)` 은 0~99까지 값을 차례대로 꺼낼 수 있는 이터레이터를 하나 만들어내는 것임
- 만약, 위처럼 연속된 숫자를 미리 만들어 값을 차례대로 사용한다면, 숫자가 적을 때는 상관없지만 많은 상황에서는 메모리 사용을 많이 하게 되므로 성능이 저하될 수 있음
- 그래서 파이썬에는 미리 값을 만들어 두는 것이 아니라 **iterator**를 생성하여 **필요할때 값을 만들어 사용하는 방식**을 사용하고 있음
- 이런 방식을 **lazy evaluation**이라고 함
- 참고로 iterator는 반복자라고 부르기도 함

## 1. 반복 가능 객체 알아보기
- 반복 가능한 객체는 말 그대로 반복할 수 있는 객체인데 우리가 흔히 사용하는 문자열, 리스트, 딕셔너리, 세트가 반복 가능한 객체임
- 즉, 요소가 여러 개 들어있고 한 번에 하나씩 꺼낼 수 있다는 말
- 객체가 반복 가능한지를 알아보기 위해서는, `__iter__` 메소드가 들어있는지 확인하면 됨
- 다음과 같이 `dir()` 메소드를 사용하여 객체의 메소드를 확인할 수 있다.

In [3]:
print(dir([1, 2, 3]))
print()
print([1, 2, 3].__iter__)

['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']

<method-wrapper '__iter__' of list object at 0x0000020D6F13E440>


- 리스트의 이터레이터를 변수에 저장한 뒤, __next__ 메소드를 호출해보면 리스트 내의 요소를 차례대로 꺼낼 수 있다.
- 리스트, 딕셔너리, 문자열, 세트도 iterable함 그러므로 실습은 생략
- range() 메소드도 역시 iterable하다

In [10]:
list_ = [1, 2, 3].__iter__()
print(list_.__next__())
print(list_.__next__())
print(list_.__next__())

print()

range_ = range(3).__iter__()
print(range_.__next__())
print(range_.__next__())
print(range_.__next__())

1
2
3

0
1
2


## 2. 반복 가능한 객체(iterable)와 이터레이터(iterator)
- iterable과 iterator는 엄연히 다름
- `for i in range(3)`의 예로 알아보자
- ![화면 캡처 2021-03-22 224001](https://user-images.githubusercontent.com/54063179/111998667-99989600-8b5f-11eb-9a0a-e85b65663a02.png)
- `range(3)`(iterable 객체)는 `__iter__` 메소드(iterator)를 가진다.
- 이때, `__iter__`는 `__next__` 메소드를 가지는데 이 `__next__`를 사용하여 객체를 차례대로 참조한다.

## 3. 이터레이터 만들기
 - `__iter__.__next__` 메서드를 구현해서 직접 이터레이터를 만들어보자
 - range() 메소드와 같이 작동하는 이터레이터이다.

In [13]:
class TempIterator:
    def __init__(self, stop):
        self.current = 0
        self.stop = stop

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.stop:
            r = self.current
            self.current += 1
            return r
        else:
            raise StopIteration


for i in TempIterator(3):
    print(i)

0
1
2


## 4. 이터레이터 언패킹
- 이터레이터는 언패킹이 가능하다
- 사실 많이 사용하는 방법이다.

In [14]:
list_ = [1, 2, 3]
a, b, c = list_
print(a)
print(b)
print(c)

1
2
3


- map함수도 이터레이터이다.

In [18]:
list_ = [1, 2, 3]

a, b, c = map(lambda x : x*2, list_)
print(a)
print(b)
print(c)

2
4
6


## 5. 인덱스로 접근할 수 있는 이터레이터 만들기
- 지금까지 `__iter__`와 `__next__`메서드를 구현하는 방식으로 이터레이터를 만들었다.
- 이번에는 `__getitem__` 메서드를 구현하여 인덱스로 접근할 수 있는 이터레이터를 만들어보자.

In [2]:
class TempIterator:
    def __init__(self, stop):
        self.stop = stop

    def __getitem__(self, index):
        if index < self.stop:
            return index
        else:
            raise IndexError

In [7]:
print(TempIterator(3)[0], TempIterator(3)[1], TempIterator(3)[2])
for i in TempIterator(3):
    print(i, end=' ')

0 1 2
0 1 2 

- 다시 정의한 TempIterator 클래스를 잘보면 `__iter__` 메소드와 `__next__` 메소드가 없어도 잘 실행됨을 볼 수 있다.
- 이것은 `__getitem__` 메소드를 작성하면 `__itet__`와 `__next__`는 생략해도 되기 때문이다.

## 6. `iter()`, `next()` 내장함수
- 이번에는 `__iter__`나 `__next__`와 같은 메소드가 아니라 파이썬에서 제공해주는 `iter()`, `next()` 내장함수를 알아보자
- `iter()`와 `next()`는 객체의 `__iter__` 메소드와 `__next__` 메소드를 호출해주는 함수이다.
- 다음의 예를 보자

In [13]:
iter_object = iter(range(3))
print(next(iter_object))
print(next(iter_object))
print(next(iter_object))

iter_object02 = range(3).__iter__()
print(iter_object02.__next__())
print(iter_object02.__next__())
print(iter_object02.__next__())

0
1
2
0
1
2


## 7. `iter()`
- `iter(호출가능한객체, 반복을끝낼값))`
- `iter()`함수에 `반복가능한객체`가 아니라 `호출가능한객체`를 입력해야 한다.
- `random.randint(0, 5)`에서 4가나오면 반복을 끝내는 예제를 실습해보자

In [28]:
import random
it = iter(lambda : random.randint(0, 5), 4)

print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))


3
5
1


StopIteration: 

- `next(it)`로 계속 난수를 생성하다보면, `StropIteration`이 만들어진다.
- 물론 if 문으로도 위의 코드를 재현할 수 있지만, if 조건문은 매번 확인해야 한다는 점에서 위 코드보다 더욱 복잡한 방법이다.

In [29]:
import random

while True:
    i = random.randint(0, 5)
    if i == 2:
        break
    print(i)

5
4
1
4
3
0


8. `next()`
- `next()`는 기본 값을 지정할 수 있다.
- `next(반복가능한객체, 기본값)`
- 여기서 기본값은 `StopIteration`이 발생하지 않고, 기본값을 출력한다는 것이다.

In [32]:
it = iter(range(3))
print(next(it, 10))
print(next(it, 10))
print(next(it, 10))
print(next(it, 10))
print(next(it, 10))

0
1
2
10
10
